In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

In [15]:
VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [16]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (16, 8, 8)     (1, 1)   (4, 4)  
LeakyReLU    (16, 8, 8)     (0.001) 
BatchNorm2d  (16, 8, 8)    
Flatten      1024          
Linear       1024          
ReLU         1024          
Linear       10            



In [17]:
fixed_model_gpu

Sequential(
  (0): Conv2d(3, 16, kernel_size=(1, 1), stride=(4, 4))
  (1): LeakyReLU(negative_slope=0.001)
  (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Flatten()
  (4): Linear(in_features=1024, out_features=1024, bias=True)
  (5): ReLU()
  (6): Linear(in_features=1024, out_features=10, bias=True)
)

In [18]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 8.3s 
val_acc: 0.468, train_acc: 0.462
